# 문제 07 유형(DataSet_07.csv 이용)

 구분자 : comma(“,”), 400 Rows, 9 Columns, UTF-8 인코딩

 대학원 진학을 위하여 어떤 항목이 중요하게 영향을 미치는지 아래 데이터로 분석하고자 한다.

 컬 럼 / 정 의 / Type
 Serial_No / 구분자 번호 / Double
 GRE / GRE 시험 성적 / Double
 TOEFL / TOEFL 시험 성적 / Double
 University_Rating / 대학 평가 그룹 (1 ~ 5) / Double
 SOP / 자기 소개서 점수 (1 ~ 5) / Double
 LOR / 추천서 점수 (1 ~ 5) / Double
 CGPA / 학부 평량 평점 (10점 만점 환산 점수) / Double
 Research / 연구 참여 경험 여부 (0 / 1) / Double
 Chance_of_Admit / 합격 가능성 / Double

In [2]:
import pandas as pd
import numpy as np

data7=pd.read_csv("C:/Users/joey0/CodeStudy/proDS/Dataset/Dataset_07.csv")
data7.columns

Index(['Serial_No', 'GRE', 'TOEFL', 'University_Rating', 'SOP', 'LOR', 'CGPA',
       'Research', 'Chance_of_Admit'],
      dtype='object')

## 1. 합격 가능성에 GRE, TOEFL, CGPA 점수 가운데 가장 영향이 큰 것이 어떤 점수인지 알아 보기 위해서 상관 분석을 수행한다.
 - 피어슨(Pearson) 상관계수 값을 구한다.
 - Chance_of_Admit와의 가장 큰 상관계수 값을 가지는 항목의 상관계수를 소수점 넷째 자리에서 반올림하여 셋째 자리까지 기술하시오.

In [3]:
var_list=['GRE', 'TOEFL', 'CGPA','Chance_of_Admit']


q1=data7[var_list].corr().drop('Chance_of_Admit')['Chance_of_Admit']
abs(q1).max() 

#정답 : 0.873

0.8732890993553003

## 2.GRE 점수의 평균 이상을 받은 그룹과 평균 미만을 받은 그룹의 CGPA 평균은 차이가 있는지 검정을 하고자 한다.
 - 적절한 검정 방법을 선택하고 양측 검정을 수행하시오 (등분산으로 가정)
 - 검정 결과, 검정통계량의 추정치를 소수점 셋째 자리에서 반올림하여 소수점 두 자리까지 기술하시오.

In [4]:
mu=data7['GRE'].mean()

q2=data7.copy()
q2['GRE_gr']=np.where(q2.GRE >= mu, 1, 0)

g1=q2[q2.GRE_gr == 1]['CGPA']
g0=q2[q2.GRE_gr == 0]['CGPA']


from scipy.stats import ttest_ind

q2_out=ttest_ind(g1, g0, equal_var=True)

q2_out.statistic

#정답 : 19.44

19.443291692470982

## 3.Chance_of_Admit 확률이 0.5를 초과하면 합격으로, 이하이면 불합격으로 구분하고 로지스틱 회귀분석을 수행하시오.
 - 원데이터만 사용하고, 원데이터 가운데 Serial_No와 Label은 모형에서 제외
 - 각 설정값은 다음과 같이 지정하고, 언급되지 않은 사항은 기본 설정값을 사용하시오
 Seed : 123
 - 로지스틱 회귀분석 수행 결과에서 로지스틱 회귀계수의 절대값이 가장 큰 변수와 그 값을 기술하시오. (로지스틱 회귀계수는 반올림하여 소수점 둘째 자리까지 / Intercept는 제외)

In [5]:
q3=data7.copy()

q3['Ch_cd']=np.where(q3.Chance_of_Admit > 0.5 , 1, 0)

from sklearn.linear_model import LogisticRegression

x_list=data7.columns.drop(['Serial_No','Chance_of_Admit'])

logit=LogisticRegression(fit_intercept=False, 
                         random_state=12, solver = 'liblinear')

logit.fit(q3[x_list], q3['Ch_cd'])

LogisticRegression(fit_intercept=False, random_state=12, solver='liblinear')

In [7]:
logit.coef_.shape
q3_out=pd.Series(logit.coef_.reshape(-1))

q3_out.index=x_list
q3_out.abs().nlargest(1)

#정답 : GCPA, 1.96

CGPA    1.955354
dtype: float64